In [2]:
from keras.layers.core import Lambda

In [13]:
from __future__ import print_function, absolute_import, division
from keras.initializers import Initializer
from keras import backend as K
from keras.engine.base_layer import Layer
from keras.engine.topology import InputSpec
import tensorflow as tf,keras
from keras.layers.merge import Concatenate
from keras.utils import conv_utils
from keras.layers import Input, MaxPooling1D,MaxPooling2D,Conv1D,Activation,Dense, activations, initializers,Conv2D
from keras.layers import Flatten, regularizers,Reshape, constraints, Dropout,Multiply,Add
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.utils import plot_model
import numpy as np
from scipy.fftpack import dct
from keras.backend.common import normalize_data_format
from keras.layers.merge import Multiply
from keras.utils import to_categorical
from keras.callbacks import LearningRateScheduler
from Gradient_Reverse_Layer import GradientReversal
from custom_layers import Conv1D_gammatone, Conv1D_zerophase
import scipy
from keras.constraints import max_norm
from keras.regularizers import l2
from keras import initializers
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda
from keras.optimizers import Adam, SGD # Nadam, Adamax

In [34]:
class MFCC(Layer): 
    def __init__(self, rank,
                 filters,
                 kernel_size,
                 output_format='signal',
                 strides=1,
                 padding='valid',
                 data_format=None,
                 dilation_rate=1,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(MFCC, self).__init__(**kwargs)
        self.rank = rank
        self.filters = filters
        self.kernel_size = conv_utils.normalize_tuple(kernel_size, rank,
                                                      'kernel_size')
        self.strides = conv_utils.normalize_tuple(strides, rank, 'strides')
        self.padding = conv_utils.normalize_padding(padding)
        self.data_format = K.normalize_data_format(data_format)
        self.kernel_initializer = kernel_initializer
        self.output_format = output_format
        self.input_spec = InputSpec(ndim=self.rank + 2)

    def build(self, input_shape):
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs '
                             'should be defined. Found `None`.')
        input_dim = input_shape[channel_axis]
        kernel_shape = self.kernel_size + (input_dim, self.filters)
    
        
        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='kernel',trainable=False)
        # Set input spec.
        self.input_spec = InputSpec(ndim=self.rank + 2,
                                    axes={channel_axis: input_dim})
        self.built = True

    def call(self, inputs):
        outputs = K.pow(K.abs(inputs),2)
        outputs = K.conv1d(
            outputs,
            self.kernel,
            strides=self.strides[0],
            padding=self.padding,
            data_format=self.data_format)
        outputs = K.log(outputs)
#         outputs = tf.signal.dct(outputs,type=2,axis=-1,norm='ortho')
        if(self.output_format=='image'):
            outputs = K.expand_dims(outputs,axis=-1)
        return outputs

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_last':
            space = input_shape[1:-1]
            new_space = []
            for i in range(len(space)):
                new_dim = conv_utils.conv_output_length(
                    space[i],
                    self.kernel_size[i],
                    padding=self.padding,
                    stride=self.strides[i])
                new_space.append(new_dim)
            if(self.output_format=='image'):
                return (input_shape[0],) + tuple(new_space) + (self.filters,) + (1,)
            else:
                return (input_shape[0],) + tuple(new_space) + (self.filters,)
            
        if self.data_format == 'channels_first':
            raise NotImplementedError("Output formate image/signal not handled")
            space = input_shape[2:]
            new_space = []
            for i in range(len(space)):
                new_dim = conv_utils.conv_output_length(
                    space[i],
                    self.kernel_size[i],
                    padding=self.padding,
                    stride=self.strides[i])
                new_space.append(new_dim)
            return (input_shape[0], self.filters) + tuple(new_space)
    
    def get_config(self):
        config = {
            'rank': self.rank,
            'filters': self.filters,
            'kernel_size': self.kernel_size,
            'strides': self.strides,
            'padding': self.padding,
            'data_format': self.data_format
        }
        base_config = super(_Conv, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [18]:
def to_colormap(x):
    import matplotlib._cm_listed as cmlist
    cm = tf.convert_to_tensor(np.array(cmlist._viridis_data))
    b = 255*(x-K.min(x))/(K.max(x)-K.min(x))
    b = K.cast(b,dtype='int64')
    wow = K.gather(cm,b)
    return K.cast(wow,dtype='float32')
def colormap_output_shape(input_shape):
    return tuple( list(input_shape)+[3] )

class TO_colormap(Layer):

    def __init__(self,**kwargs):
        
        super(TO_colormap, self).__init__(**kwargs)
        import matplotlib._cm_listed as cmlist
        self.cm = tf.convert_to_tensor(np.array(cmlist._viridis_data))
        self.input_spec = InputSpec(min_ndim=3)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernelA = self.add_weight(name='kernelA',
                                       shape=(input_shape[1], 3),
                                       initializer='uniform')
        
        self.built = True
        super(TO_colormap, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        b = 255*(x-K.min(x))/(K.max(x)-K.min(x))
        b = K.cast(b,dtype='int64')
        wow = K.gather(self.cm,b)
        return K.cast(wow,dtype='float32')
        
    def compute_output_shape(self, input_shape):
        return tuple( list(input_shape)+[3] )

In [19]:
def mfcc_kernel_init(shape, dtype=K.floatx()):
    (kernel_size,in_channels,out_channels) = shape
    if(in_channels!=out_channels):
        raise ValueError("Input and Output Channels must be same. Got {0} input channels and {0} output channels".format(in_channels,out_channels))
    mat = K.eye(in_channels,dtype=dtype)
    mat_n = [mat for i in range(kernel_size)]
    return K.stack(mat_n)
class Freq_Init(Initializer):
    def __init__(self, minf=0., maxf=500):
        self.minf = minf
        self.maxf = maxf
    def __call__(self, shape, dtype=K.floatx()):
        (kernel_size,in_channels) = shape
        start = self.hz2mel(self.minf)
        end = self.hz2mel(self.maxf)
        n = ( end-start)/(kernel_size-1)
        mel =  K.expand_dims(K.arange(start,end+1,n,dtype=dtype),axis=1)
        return self.mel2hz(mel)
    def hz2mel(self,hz):
        return 2595 * (K.log(1.0+(hz*1.0)/700.)/K.log(10.0))
    def mel2hz(self,mel):
        return 700*(10**(mel/2595.0)-1)
    def get_config(self):
        return {
            'minf': self.minf,
            'maxf': self.maxf
        }
def hz2mel(hz):
    return 2595 * np.log10(1+hz/700.)
def mel2hz(mel):
    return 700*(10**(mel/2595.0)-1)
def erb(f):
    return 24.7*(4.37*10**-3*f+1)
def beta_init(shape, dtype=K.floatx()):
    (kernel_size,in_channels) = shape
    beta_weights = tf.convert_to_tensor(np.ones((kernel_size,1))*100,dtype=K.floatx())
    return beta_weights

In [20]:
def Network(kernel_size=5,fs=1000,winlen=0.025,winstep=0.01,filters=26,random_seed=1,padding='valid',bias=False,
           lr=0.0012843784,lr_decay=0.0001132885,subsam=2,num_filt=(26,32),num_dense=20,trainable=True,batch_size=1024,
           l2_reg=0.0,l2_reg_dense=0.0,bn_momentum=0.99,dropout_rate=0.5,dropout_dense=0.0,eps = 1.1e-5,maxnorm=10000,
           activation_function='relu'):
    input = Input(shape=(2500, 1))
    t = Conv1D_gammatone(kernel_size=81, strides=1,filters=filters,
                         fsHz=fs,use_bias=False,padding='same',
                         fc_initializer=Freq_Init(minf=50.0,maxf=fs/2),
                         amp_initializer=initializers.constant(10**4),
                        beta_initializer=beta_init,name="gamma"
                        )(input)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = MFCC(rank = 1,filters=filters,kernel_size=int(winlen*fs),output_format='signal',strides=int(winstep*fs),
              kernel_initializer = mfcc_kernel_init, name="mfcc")(t)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    
    t = branch(t,num_filt,kernel_size,random_seed,padding,bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)

    t = res_block(t,32,kernel_size,1,'same',random_seed,bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)

    t = res_block(t,64,kernel_size,2,'same',random_seed,bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
    t = Flatten()(t)
    t = Dense(20,
                   activation=activation_function,
                   kernel_initializer=initializers.he_normal(seed=random_seed),
                   use_bias=bias,
                   kernel_constraint=max_norm(maxnorm),
                   kernel_regularizer=l2(l2_reg_dense),
                   name = 'class_dense')(t)
    t = Dense(2, activation='softmax', name="class")(t)
    opt = SGD(lr=.001,decay=.001)
    
    model = Model(inputs=input, outputs=t)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [21]:
kernel_size=5
fs=2000
winlen=0.025
winstep=0.01
filters=26

In [9]:
model = Network(kernel_size,fs,winlen,winstep,filters)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2500, 1)      0                                            
__________________________________________________________________________________________________
gamma (Conv1D_gammatone)        (None, 2500, 26)     79          input_1[0][0]                    
__________________________________________________________________________________________________
mfcc (MFCC)                     (None, 123, 26)      33800       gamma[0][0]                      
__________________________________________________________________________________________________
conv1d_1 (Conv1D)    

In [15]:
plot_model(model, to_file='mfcc_res.png', show_shapes=True)

NameError: name 'model' is not defined

In [22]:
def zeropad_len(x):
    return x[:,:-1,:]
def zeropad_len_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 3
    shape[1] -= 1
    return tuple(shape)
def zeropad(x):
    y = K.zeros_like(x)
    return K.concatenate([x, y], axis=2)
def zeropad_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 3
    shape[2] *= 2
    return tuple(shape)
def res_block(input_tensor,num_filt,kernel_size,stride,padding,random_seed,bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable,cat=True):

    t = Conv1D(num_filt, kernel_size=kernel_size,
                kernel_initializer=initializers.he_normal(seed=random_seed),
                padding=padding,
                strides=stride,
                use_bias=bias,
                kernel_constraint=max_norm(maxnorm),
                trainable=trainable,
                kernel_regularizer=l2(l2_reg))(input_tensor)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Activation(activation_function)(t)
    t = Dropout(rate=dropout_rate, seed=random_seed)(t)
    t = Conv1D(num_filt, kernel_size=kernel_size,
                kernel_initializer=initializers.he_normal(seed=random_seed),
                padding=padding,
                strides=1,
                use_bias=bias,
                kernel_constraint=max_norm(maxnorm),
                trainable=trainable,
                kernel_regularizer=l2(l2_reg))(t)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Activation(activation_function)(t)
    t = Dropout(rate=dropout_rate, seed=random_seed)(t)
    
    p = MaxPooling1D(pool_size=stride)(input_tensor)
    if(stride>1):
        if(cat):
            p = Lambda(zeropad, output_shape=zeropad_output_shape)(p)
        if(t.shape[1]!=p.shape):
            t = Lambda(zeropad_len, output_shape=zeropad_len_output_shape)(t)

    t = Add()([t,p])
    return t
def branch(input_tensor,num_filt,kernel_size,random_seed,padding,bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable):

    num_filt1, num_filt2 = num_filt
    t = Conv1D(num_filt1, kernel_size=kernel_size,
                kernel_initializer=initializers.he_normal(seed=random_seed),
                padding=padding,
                use_bias=bias,
                kernel_constraint=max_norm(maxnorm),
                trainable=trainable,
                kernel_regularizer=l2(l2_reg))(input_tensor)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Activation(activation_function)(t)
    t = Dropout(rate=dropout_rate, seed=random_seed)(t)
    t = Conv1D(num_filt2, kernel_size=kernel_size,
               kernel_initializer=initializers.he_normal(seed=random_seed),
               padding=padding,
               use_bias=bias,
               trainable=trainable,
               kernel_constraint=max_norm(maxnorm),
               kernel_regularizer=l2(l2_reg))(t)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Activation(activation_function)(t)
    t = Dropout(rate=dropout_rate, seed=random_seed)(t)
    
    return t

In [23]:
def zeropad_len_Ax1(x):
    print(x.shape)
    return x[:,:-1,:,:]
def zeropad_len_Ax2(x):
    return x[:,:,:-1,:]
def zeropad_len_output_shape_Ax1(input_shape):
    shape = list(input_shape)
    assert len(shape) == 4
    shape[1] -= 1
    return tuple(shape)
def zeropad_len_output_shape_Ax2(input_shape):
    shape = list(input_shape)
    assert len(shape) == 4
    shape[2] -= 1
    return tuple(shape)
def zeropad2d(x):
    y = K.zeros_like(x)
    return K.concatenate([x, y], axis=3)
def zeropad_output_shape2d(input_shape):
    shape = list(input_shape)
    assert len(shape) == 4
    shape[-1] *= 2
    return tuple(shape)
def branch2d(input_tensor,num_filt,kernel_size,random_seed,padding,bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable):
    
    pad1,pad2 = padding
    num_filt1, num_filt2 = num_filt
    t = Conv2D(num_filt1, kernel_size=kernel_size,
                kernel_initializer=initializers.he_normal(seed=random_seed),
                padding=pad1,
                use_bias=bias,
                kernel_constraint=max_norm(maxnorm),
                trainable=trainable,
                kernel_regularizer=l2(l2_reg))(input_tensor)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Activation(activation_function)(t)
    t = Dropout(rate=dropout_rate, seed=random_seed)(t)
    t = Conv2D(num_filt2, kernel_size=kernel_size,
               kernel_initializer=initializers.he_normal(seed=random_seed),
               padding=pad2,
               use_bias=bias,
               trainable=trainable,
               kernel_constraint=max_norm(maxnorm),
               kernel_regularizer=l2(l2_reg))(t)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Activation(activation_function)(t)
    t = Dropout(rate=dropout_rate, seed=random_seed)(t)
    
    return t
def res_block2d(input_tensor,num_filt,kernel_size,stride,padding,random_seed,bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable,cat=True):

    t = Conv2D(num_filt, kernel_size=kernel_size,
                kernel_initializer=initializers.he_normal(seed=random_seed),
                padding=padding,
                strides=stride,
                use_bias=bias,
                kernel_constraint=max_norm(maxnorm),
                trainable=trainable,
                kernel_regularizer=l2(l2_reg))(input_tensor)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Activation(activation_function)(t)
    t = Dropout(rate=dropout_rate, seed=random_seed)(t)
    t = Conv2D(num_filt, kernel_size=kernel_size,
                kernel_initializer=initializers.he_normal(seed=random_seed),
                padding=padding,
                strides=1,
                use_bias=bias,
                kernel_constraint=max_norm(maxnorm),
                trainable=trainable,
                kernel_regularizer=l2(l2_reg))(t)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Activation(activation_function)(t)
    t = Dropout(rate=dropout_rate, seed=random_seed)(t)
    
    p = MaxPooling2D(pool_size=stride)(input_tensor)
    if(stride>1):
        if(cat):
            p = Lambda(zeropad2d, output_shape=zeropad_output_shape2d)(p)
#         print("T", t.shape, "P", p.shape)
        if(t.shape[1]!=p.shape[1]):
            t = Lambda(zeropad_len_Ax1, output_shape=zeropad_len_output_shape_Ax1)(t)
        if(t.shape[2]!=p.shape[2]):
            t = Lambda(zeropad_len_Ax2, output_shape=zeropad_len_output_shape_Ax2)(t)
#         print("T", t.shape, "P", p.shape, "PORe")
    t = Add()([t,p])
    return t

In [24]:
help(Reshape)

Help on class Reshape in module keras.layers.core:

class Reshape(keras.engine.base_layer.Layer)
 |  Reshape(target_shape, **kwargs)
 |  
 |  Reshapes an output to a certain shape.
 |  
 |  # Arguments
 |      target_shape: target shape. Tuple of integers.
 |          Does not include the batch axis.
 |  
 |  # Input shape
 |      Arbitrary, although all dimensions in the input shaped must be fixed.
 |      Use the keyword argument `input_shape`
 |      (tuple of integers, does not include the batch axis)
 |      when using this layer as the first layer in a model.
 |  
 |  # Output shape
 |      `(batch_size,) + target_shape`
 |  
 |  # Example
 |  
 |  ```python
 |      # as first layer in a Sequential model
 |      model = Sequential()
 |      model.add(Reshape((3, 4), input_shape=(12,)))
 |      # now: model.output_shape == (None, 3, 4)
 |      # note: `None` is the batch dimension
 |  
 |      # as intermediate layer in a Sequential model
 |      model.add(Reshape((6, 2)))
 |     

In [53]:
def Network2D(kernel_size=5,fs=1000,winlen=0.025,winstep=0.01,filters=26,random_seed=1,padding='valid',bias=False,
           lr=0.0012843784,lr_decay=0.0001132885,subsam=2,num_filt=(16,32),num_dense=20,trainable=True,batch_size=1024,
           l2_reg=0.0,l2_reg_dense=0.0,bn_momentum=0.99,dropout_rate=0.5,dropout_dense=0.0,eps = 1.1e-5,maxnorm=10000,
           activation_function='relu'):
    input = Input(shape=(2500, 1))
    t = Conv1D_gammatone(kernel_size=81,strides=1,filters=filters,
                         fsHz=fs,use_bias=False,padding='same',
                         fc_initializer=Freq_Init(minf=50.0,maxf=fs/2),
                         amp_initializer=initializers.constant(10**4),
                        beta_initializer=beta_init,name="gamma"
                        )(input)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = MFCC(rank = 1,filters=filters,kernel_size=int(winlen*fs),output_format='signal',strides=int(winstep*fs),
              kernel_initializer = mfcc_kernel_init, name="mfcc")(t)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Reshape(target_shape=(-1,filters,1))(t)
#     t = Lambda(to_colormap,output_shape = colormap_output_shape)(t)
#     t = TO_colormap()(t)
    t = branch2d(t,num_filt,kernel_size,random_seed,('valid','valid'),bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
    t = MaxPooling2D((2,1))(t)
    t = branch2d(t,(32,64),(kernel_size,3),random_seed,('valid','valid'),bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
    t = MaxPooling2D((2,1))(t)
    t = branch2d(t,(64,128),kernel_size,random_seed,('valid','valid'),bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
    t = MaxPooling2D((2,1))(t)
    t = branch2d(t,(128,256),kernel_size,random_seed,('same','same'),bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
#     t = res_block2d(t,32,5,1,'valid',random_seed,bias,maxnorm,l2_reg,
#            eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)

#     t = res_block2d(t,64,5,2,'same',random_seed,bias,maxnorm,l2_reg,
#            eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
    
#     t = res_block2d(t,128,5,2,'same',random_seed,bias,maxnorm,l2_reg,
#            eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
#     t = MaxPooling2D((2,1))(t)
    t = Flatten()(t)
    t = Dense(20,
                   activation=activation_function,
                   kernel_initializer=initializers.he_normal(seed=random_seed),
                   use_bias=bias,
                   kernel_constraint=max_norm(maxnorm),
                   kernel_regularizer=l2(l2_reg_dense),
                   name = 'class_dense')(t)
    t = Dense(2, activation='softmax', name="class")(t)
    opt = Adam(lr=.001,decay=.001,epsilon=eps)
    
    model = Model(inputs=input, outputs=t)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [54]:
# from HeartCepNet import heartnet2D
kernel_size=5
fs=2000
winlen=0.025
winstep=0.01
filters=26
# model = heartnet2D(kernel_size,fs,winlen,winstep,filters)
model = Network2D(kernel_size,fs,winlen,winstep,filters)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 2500, 1)           0         
_________________________________________________________________
gamma (Conv1D_gammatone)     (None, 2500, 26)          79        
_________________________________________________________________
batch_normalization_29 (Batc (None, 2500, 26)          104       
_________________________________________________________________
mfcc (MFCC)                  (None, 123, 26)           33800     
_________________________________________________________________
batch_normalization_30 (Batc (None, 123, 26)           104       
_________________________________________________________________
reshape_5 (Reshape)          (None, 123, 26, 1)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 119, 22, 16)       400       
__________

In [55]:
# plot_model(model, to_file='mfcc_res2D.png', show_shapes=True)

In [56]:
import numpy as np
from keras.utils import to_categorical

In [57]:
x = np.random.rand(10,2500,1)
y = np.ones((10,1))
y = to_categorical(y)
x.shape,y.shape

((10, 2500, 1), (10, 2))

In [58]:
model.fit(x,y,epochs=10)

Epoch 1/10
10/10 [==============================] - 2s 205ms/step - loss: 1.1416 - acc: 0.3000
Epoch 2/10
10/10 [==============================] - 0s 1ms/step - loss: 6.7836e-05 - acc: 1.0000
Epoch 3/10
10/10 [==============================] - 0s 1ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 4/10
10/10 [==============================] - 0s 1ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 5/10
10/10 [==============================] - 0s 1ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 6/10
10/10 [==============================] - 0s 1ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 7/10
10/10 [==============================] - 0s 1ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 8/10
10/10 [==============================] - 0s 2ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 9/10
10/10 [==============================] - 0s 1ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 10/10
10/10 [==============================] - 0s 1ms/step - loss: 1.1921e-07 - acc: 1.0000
